In [2]:
!pip show tensorflow



Name: tensorflow
Version: 2.20.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: D:\shri\RealtyAI\.venv\Lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google_pasta, grpcio, h5py, keras, libclang, ml_dtypes, numpy, opt_einsum, packaging, protobuf, requests, setuptools, six, tensorboard, termcolor, typing_extensions, wrapt
Required-by: 


In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.20.0


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model

print("🔍 VERIFYING YOUR EXACT DATA...")

# Load your data
X_train = np.load('../data/spacenet/X_train.npy')
y_train = np.load('../data/spacenet/y_train.npy')
X_val = np.load('../data/spacenet/X_val.npy')
y_val = np.load('../data/spacenet/y_val.npy')

print("📊 YOUR EXACT DATA SHAPES:")
print(f"X_train: {X_train.shape}")  # Should be (996, 256, 256, 4)
print(f"y_train: {y_train.shape}")  # Should be (996, 256, 256)
print(f"X_val: {X_val.shape}")      # Should be (249, 256, 256, 4) 
print(f"y_val: {y_val.shape}")      # Should be (249, 256, 256)

print("🎯 DATA PROPERTIES:")
print(f"Image range: [{X_train.min():.3f}, {X_train.max():.3f}]")
print(f"Mask unique values: {np.unique(y_train)}")
print(f"Data type - X: {X_train.dtype}, y: {y_train.dtype}")

🔍 VERIFYING YOUR EXACT DATA...
📊 YOUR EXACT DATA SHAPES:
X_train: (996, 256, 256, 4)
y_train: (996, 256, 256)
X_val: (249, 256, 256, 4)
y_val: (249, 256, 256)
🎯 DATA PROPERTIES:
Image range: [0.000, 1.000]
Mask unique values: [0. 1.]
Data type - X: float32, y: float32


In [5]:
print("\n BUILDING UNET MODEL FOR YOUR EXACT DATA...")

def build_unet(input_shape=(256, 256, 4)):
    """UNet for 256x256x4 input -> 256x256x1 output (building masks)"""
    
    inputs = Input(shape=input_shape)
    
    # Encoder (Downsampling)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)  # 256x256 -> 128x128
    
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)  # 128x128 -> 64x64
    
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)  # 64x64 -> 32x32
    
    # Bottleneck
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    
    # Decoder (Upsampling)
    u5 = UpSampling2D((2, 2))(c4)  # 32x32 -> 64x64
    u5 = concatenate([u5, c3])
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(c5)
    
    u6 = UpSampling2D((2, 2))(c5)  # 64x64 -> 128x128
    u6 = concatenate([u6, c2])
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(c6)
    
    u7 = UpSampling2D((2, 2))(c6)  # 128x128 -> 256x256
    u7 = concatenate([u7, c1])
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(c7)
    
    # Output: 1 channel (building vs not building)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)
    
    model = Model(inputs, outputs, name='UNet_Building_Segmentation')
    return model

# Build model
model = build_unet(input_shape=X_train.shape[1:])
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', 'mean_squared_error']
)

print("✅ MODEL BUILT SUCCESSFULLY!")
print(f"Input shape:  {model.input_shape}")
print(f"Output shape: {model.output_shape}")
print(f"Total parameters: {model.count_params():,}")

# Show model architecture
model.summary()


 BUILDING UNET MODEL FOR YOUR EXACT DATA...
✅ MODEL BUILT SUCCESSFULLY!
Input shape:  (None, 256, 256, 4)
Output shape: (None, 256, 256, 1)
Total parameters: 7,783,489


Model: "UNet_Building_Segmentation"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      2,368 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │    590,080 │ conv2d_4[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 32, 32,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 32, 32,    │  1,180,160 │ max_pooling2d_2[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 32, 32,    │  2,359,808 │ conv2d_6[0][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 64, 64,    │          0 │ conv2d_7[0][0]    │
│ (UpSampling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64, 64,    │          0 │ up_sampling2d[0]… │
│ (Concatenate)       │ 768)              │            │ conv2d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 64, 64,    │  1,769,728 │ concatenate[0][0] │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 64, 64,    │    590,080 │ conv2d_8[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_1     │ (None, 128, 128,  │          0 │ conv2d_9[0][0]  

 Total params: 7,783,489 (29.69 MB)

 Trainable params: 7,783,489 (29.69 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
print("🚀 OPTIMIZING TRAINING SPEED...")

# Reduce batch size and simplify model for faster training
def build_faster_unet(input_shape=(256, 256, 4)):
    """Lighter UNet for faster training"""
    
    inputs = Input(shape=input_shape)
    
    # Smaller architecture
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    
    # Bottleneck
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    
    # Decoder
    u4 = UpSampling2D((2, 2))(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(u4)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(c4)
    
    u5 = UpSampling2D((2, 2))(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(u5)
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(c5)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)
    
    model = Model(inputs, outputs)
    return model

# Build faster model
fast_model = build_faster_unet(input_shape=X_train.shape[1:])
fast_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("✅ Faster model built!")
print(f"Parameters: {fast_model.count_params():,} (vs 7.8M before)")

# Train with smaller batch size
print("🎯 STARTING FASTER TRAINING...")
history = fast_model.fit(
    X_train, y_train,
    batch_size=4,  # Smaller batch for memory
    epochs=15,     # Fewer epochs
    validation_data=(X_val, y_val),
    verbose=1
)

🚀 OPTIMIZING TRAINING SPEED...
✅ Faster model built!
Parameters: 471,841 (vs 7.8M before)
🎯 STARTING FASTER TRAINING...
Epoch 1/15
249/249 ━━━━━━━━━━━━━━━━━━━━ 312s 1s/step - accuracy: 0.9240 - loss: 0.2669 - val_accuracy: 0.9199 - val_loss: 0.2658
Epoch 2/15
249/249 ━━━━━━━━━━━━━━━━━━━━ 304s 1s/step - accuracy: 0.9254 - loss: 0.2490 - val_accuracy: 0.9199 - val_loss: 0.2533
Epoch 3/15
249/249 ━━━━━━━━━━━━━━━━━━━━ 322s 1s/step - accuracy: 0.9260 - loss: 0.2394 - val_accuracy: 0.9213 - val_loss: 0.2412
Epoch 4/15
249/249 ━━━━━━━━━━━━━━━━━━━━ 319s 1s/step - accuracy: 0.9264 - loss: 0.2396 - val_accuracy: 0.9227 - val_loss: 0.2453
Epoch 5/15
249/249 ━━━━━━━━━━━━━━━━━━━━ 302s 1s/step - accuracy: 0.9269 - loss: 0.2382 - val_accuracy: 0.9205 - val_loss: 0.2533
Epoch 6/15
249/249 ━━━━━━━━━━━━━━━━━━━━ 296s 1s/step - accuracy: 0.9273 - loss: 0.2348 - val_accuracy: 0.9216 - val_loss: 0.2433
Epoch 7/15
249/249 ━━━━━━━━━━━━━━━━━━━━ 324s 1s/step - accuracy: 0.9272 - loss: 0.2309 - val_accuracy: 0.9

In [1]:
import tensorflow.keras.backend as K

def iou(y_true, y_pred, smooth=1e-6):
    y_pred = K.round(y_pred)
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - intersection
    return K.mean((intersection + smooth) / (union + smooth))

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_pred = K.round(y_pred)
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    denom = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean((2. * intersection + smooth) / (denom + smooth))

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

# --- Define metrics ---
def iou(y_true, y_pred, smooth=1e-6):
    y_pred = K.round(y_pred)
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - intersection
    return K.mean((intersection + smooth) / (union + smooth))

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_pred = K.round(y_pred)
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    denom = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean((2. * intersection + smooth) / (denom + smooth))

# --- Build lighter UNet ---
def build_faster_unet(input_shape=(256, 256, 4)):
    inputs = Input(shape=input_shape)
    
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    
    u4 = UpSampling2D((2, 2))(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(u4)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(c4)
    
    u5 = UpSampling2D((2, 2))(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(u5)
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(c5)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)
    return Model(inputs, outputs)

# --- Load data ---
print("Loading data...")
X_train = np.load('../data/spacenet/X_train.npy')
y_train = np.load('../data/spacenet/y_train.npy')
X_val = np.load('../data/spacenet/X_val.npy')
y_val = np.load('../data/spacenet/y_val.npy')

# --- Build and compile model ---
print("Building model...")
fast_model = build_faster_unet(input_shape=X_train.shape[1:])
fast_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', iou, dice_coef]
)

print("✅ Model compiled with IoU and Dice!")
print("Metrics:", fast_model.metrics_names)

Loading data...
Building model...
✅ Model compiled with IoU and Dice!
Metrics: ['loss', 'compile_metrics']


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model

# --- Register custom metrics properly ---
@tf.keras.utils.register_keras_serializable()
def iou(y_true, y_pred, smooth=1e-6):
    y_pred = tf.round(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    union = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3]) - intersection
    return tf.reduce_mean((intersection + smooth) / (union + smooth))

@tf.keras.utils.register_keras_serializable()
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_pred = tf.round(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    denom = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3])
    return tf.reduce_mean((2. * intersection + smooth) / (denom + smooth))

# --- Build lighter UNet ---
def build_faster_unet(input_shape=(256, 256, 4)):
    inputs = Input(shape=input_shape)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    
    u4 = UpSampling2D((2, 2))(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(u4)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(c4)
    
    u5 = UpSampling2D((2, 2))(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(u5)
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(c5)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)
    return Model(inputs, outputs)

# --- Load data ---
print("Loading data...")
X_train = np.load('../data/spacenet/X_train.npy')
y_train = np.load('../data/spacenet/y_train.npy')
X_val = np.load('../data/spacenet/X_val.npy')
y_val = np.load('../data/spacenet/y_val.npy')

# --- Build and compile model ---
print("Building model...")
fast_model = build_faster_unet(input_shape=X_train.shape[1:])
fast_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', iou, dice_coef]
)

print("✅ Model compiled with IoU and Dice!")
print("Metrics:", [m.name if hasattr(m, 'name') else m.__name__ for m in fast_model.compiled_metrics._metrics])

Loading data...
Building model...
✅ Model compiled with IoU and Dice!


AttributeError: 'DeprecatedCompiledMetric' object has no attribute '_metrics'